In [1]:
# Imports

In [2]:
import os
data_directory = "data/data_with_goggle/"
files = os.listdir(data_directory)
print(files[:1])
print(len(files))

['gyro_sober_ys_s8_3.txt']
150


In [3]:
gyro_files = [file for file in files if file.startswith('gyro')]
accel_files = [file for file in files if file.startswith('accel')]
header_names = [file[5:] for file in gyro_files] # remove "gyro_"
print(len(gyro_files))
print(len(accel_files))

75
75


In [4]:
from datetime import datetime, timedelta
def read_filedata(filename):
    """
    Returns list of tuple consisting - `timestamp`, `x`, `y`, `z`
    [
    (`timestamp`,`x`,`y`,`z`), ...
    ]
    """
    with open(filename) as f:
        lines = f.readlines()
        # line.strip().split()
        # will give `timestamp` `x` `y` `z`
        lines = [line.strip().split() for line in lines]
        
        # Some data has less datapoints because the output was not flushed
        # So we only filter out normal ones
        lines = [line for line in lines if len(line) == 4]
        lines = [(datetime.fromtimestamp(float(ts)/1000), float(x), float(y), float(z)) for (ts, x, y, z) in lines]
    return lines

In [5]:
def preprocess(data, seconds=5):
    print("original data points: ", len(data))
    # Remove duplicates
    no_dup = []
    for idx, line in enumerate(data):
        ts, x, y, z = line
        if 0 <= idx-1 and ts == data[idx-1][0]:
            continue
        else:
            no_dup.append(line)
    print("Data points after removing duplicates", len(no_dup))
    
    # Remove n seconds from start / end
    delta = timedelta(days=0,seconds=seconds)
    start_time = no_dup[0][0] + delta
    end_time = no_dup[-1][0] - delta
    no_dup = [(ts, x, y, z) for (ts, x, y, z) in no_dup if start_time <= ts <= end_time]
    print("Data points after removing edges", len(no_dup))
    
    print("Time between" , start_time.strftime('%Y-%m-%d %H:%M:%S'), "-",  end_time.strftime('%Y-%m-%d %H:%M:%S'))
    print("Time took ", (end_time - start_time).total_seconds(), "seconds")
    return no_dup

In [6]:
def plot_graph(data):
    plt.plot(list(map(lambda x:x[0], data)), list(map(lambda x:x[1], data)))
    plt.gcf().autofmt_xdate()
    # Erase y axis
    plt.gca().get_yaxis().set_visible(False)
    plt.show()

In [7]:
from matplotlib import pyplot as plt
#from matplotlib import style 
#style.use('ggplot')

for name in header_names:
    color, person, phone, number = name[:-4].split('_') # remove ".txt" and split by '_'
    print(color, person, phone, number)
    gyro = read_filedata(data_directory + 'gyro_' + name)
    accel = read_filedata(data_directory + 'accel_' + name)
    
    print("gyroscope data points : ", len(gyro))
    print("accelerometer data points : ", len(accel))
    gyro = preprocess(gyro)
    accel = preprocess(accel)
    break

sober ys s8 3
gyroscope data points :  34672
accelerometer data points :  34672
original data points:  34672
Data points after removing duplicates 20640
Data points after removing edges 17657
Time between 2018-05-11 22:25:00 - 2018-05-11 22:25:59
Time took  59.384 seconds
original data points:  34672
Data points after removing duplicates 20416
Data points after removing edges 17450
Time between 2018-05-11 22:25:00 - 2018-05-11 22:25:59
Time took  59.384 seconds
(datetime.datetime(2018, 5, 11, 22, 25, 0, 368000), -0.8296723, -2.2058234, 0.58078504)


In [8]:
import numpy as np

In [9]:
fft_x = np.fft.fft(list(map(lambda x:x[1], gyro)))
fft_y = np.fft.fft(list(map(lambda x:x[2], gyro)))
fft_z = np.fft.fft(list(map(lambda x:x[3], gyro)))

In [10]:
print(len(fft_x))
print(len(fft_y))
print(len(fft_z))
print(len(gyro))
print(fft_x)

17657
17657
17657
17657
[-1538.88936099  +0.j            49.16281661 +60.86386324j
   -53.81400573+158.12532536j ...,   209.06277182  +7.42157856j
   -53.81400573-158.12532536j    49.16281661 -60.86386324j]


In [11]:
fft_x = np.fft.fft(list(map(lambda x:x[1], accel)))
fft_y = np.fft.fft(list(map(lambda x:x[2], accel)))
fft_z = np.fft.fft(list(map(lambda x:x[3], accel)))

In [12]:
print(len(fft_x))
print(len(fft_y))
print(len(fft_z))
print(len(accel))

17450
17450
17450
17450
